In [1]:
pip install ipynb

Note: you may need to restart the kernel to use updated packages.


In [2]:
from ipynb.fs.full.Oscar_information_3 import nominees_df

API request successful! Status code: 200
Number of nominees retrieved: 5
   id                 name  edition  year
0   1   1st Academy Awards        1  1927
1   2   2nd Academy Awards        2  1928
2   3   3rd Academy Awards        3  1929
3   4   4th Academy Awards        4  1930
4   5   5th Academy Awards        5  1931
5   6   6th Academy Awards        6  1932
6   7   7th Academy Awards        7  1934
7   8   8th Academy Awards        8  1935
8   9   9th Academy Awards        9  1936
9  10  10th Academy Awards       10  1937

    edition  year
0        64  1991
1        65  1992
2        66  1993
3        67  1994
4        68  1995
5        69  1996
6        70  1997
7        88  2015
8        89  2016
9        90  2017
10       91  2018
11       92  2019
12       93  2020
13       94  2021
Trying nominees URL for Actor In A Leading Role in 1991: https://theawards.vercel.app/api/oscars/editions/64/categories/4576/nominees
Trying nominees URL for Actress In A Leading Role in 1991: h

In [3]:
import pandas as pd


In [4]:
#Creating a list of urls for 1991 - 1997 Movies
nominees_df_unique = nominees_df[['year', 'movie']].drop_duplicates() #dropping duplicates
movies = nominees_df_unique.values 

#preparing empty lists to store the urls by year
url_91 = []
url_92 = []
url_93 = []
url_94 = []
url_95 = []
url_96 = []
url_97 = []

def titles(movie): #function that replaces the spaces in the movie titles with dashes for webscraping 
    for i in movie:
        return movie.replace(" ", "-") 

#Main loop that builds URLS
for i in movies:
    year, movie_name = i
    url_ind = 'https://www.the-numbers.com/movie/'+titles(movie_name)+'#tab=box-office' #testing url
    if year == 1991:
        url_91.append(url_ind)
    elif year == 1992:
        url_92.append(url_ind)
    elif year == 1993:
        url_93.append(url_ind)
    elif year == 1994:
        url_94.append(url_ind)
    elif year == 1995:
        url_95.append(url_ind)
    elif year == 1996:
        url_96.append(url_ind)
    elif year == 1997:
        url_97.append(url_ind)

In [5]:
#Creating a list of urls for 2015 - 2016 Movies
nominees_df_unique = nominees_df[['year', 'movie']].drop_duplicates()
movies = nominees_df_unique.values 

#preparing empty lists to store the urls by year
url_15 = []
url_16 = []
url_17 = []
url_18 = []
url_19 = []
url_20 = []
url_21 = []

def titles(movie):  #function that replaces the spaces in the movie titles with dashes for webscraping
    for i in movie:
        return movie.replace(" ", "-")

#Main loop that builds URLS
for i in movies:
    year, movie_name = i
    url_ind = 'https://www.the-numbers.com/movie/'+titles(movie_name)+'#tab=box-office' #testing url
    # print(url_ind)
    if year == 2015:
        url_15.append(url_ind)
    elif year == 2016:
        url_16.append(url_ind)
    elif year == 2017:
        url_17.append(url_ind)
    elif year == 2018:
        url_18.append(url_ind)
    elif year == 2019:
        url_19.append(url_ind)
    elif year == 2020:
        url_20.append(url_ind)
    elif year == 2021:
        url_21.append(url_ind)
    # url.append(url_ind)

# Class 

In [6]:
#Entire class made with the aid of ChatGPT

import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
import random
import time

class BoxOfficeScraper: 
    
    def __init__(self, urls, retries=3, delay=3):
        """
        urls: list of movie URLs (e.g., url_91, url_92, etc.)
        retries: number of fetch retries
        delay: delay between retries
        """
        self.urls = list(dict.fromkeys(urls))  # remove duplicates
        self.retries = retries
        self.delay = delay
        self.user_agents = [
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7)",
            "Mozilla/5.0 (X11; Linux x86_64)",
            "Mozilla/5.0 (iPhone; CPU iPhone OS 14_0 like Mac OS X)",
        ]

    def fetch_page(self, url): 
        """Fetch page with retries and random user-agent."""
        for attempt in range(self.retries):
            try:
                headers = {"User-Agent": random.choice(self.user_agents)} # Randomly choose a User-Agent so the server doesn't block you
                resp = requests.get(url, headers=headers, timeout=40) # Try requesting the URL with a 40-second timeout
                resp.raise_for_status() # raise exception if there is an error 
                return BeautifulSoup(resp.text, "html.parser") # If request was successful, parse HTML with BeautifulSoup and return it
            except Exception as error:  # If this was the FINAL url attempt, re-raise the error so the caller knows it failed
                if attempt == self.retries - 1:
                    raise
                time.sleep(self.delay) # Otherwise, wait a bit before retrying

    @staticmethod # This method does not take (self). behaves like a regular function
    def find_weekend_table(soup):
        """Find 8-column Weekend Box Office Performance table."""
        for h2 in soup.find_all("h2"): 
            if "Weekend Box Office Performance" in h2.get_text():
                for table in h2.find_all_next("table"):  # Look at every <table> that comes after this heading
                    table_headers = table.find_all("th") # Get all <th> elements (header cells) in the table
                    if len(table_headers) == 8: # If table has exactly 8 headers, it's likely the correct table
                        return table
                    first_table_row = table.find("tr") # Fallback: check the first row in case headers are <td> instead of <th>
                    if  first_table_row and len(first_table_row.find_all("td")) == 8: # If first row exists and has 8 <td> cells, it's the correct table
                        return table
        return None # If no table with 8 columns is found, return None

    @staticmethod
    def extract_table_to_df(table):
        """Convert table to data frame and clean numeric fields."""
        rows = []  # Will store all the rows of data
        for table_row in table.find_all("tr"):  # Loop through all table rows (<tr>) in the HTML table
            if table_row.find_all("th"): # Skip header rows that use <th>
                continue
            cells = [table_data.get_text(strip=True) for table_data in table_row.find_all("td")] # Extract text from all <td> cells in this row, stripping whitespace
            if len(cells) == 8:  # Only include rows with exactly 8 cells (matches weekend box office table)
                rows.append(cells) #add this row to the data frame
        if not rows: #if no valid rows were found
            return None
            
        # Define column names for the DataFrame
        columns = ["Date", "Rank", "Gross", "% Change", "Theaters",
                   "Per Theater", "Total Gross", "Week"]
        
        df = pd.DataFrame(rows, columns=columns)

        # List of columns that contain money values
        money_cols = ["Gross", "Per Theater", "Total Gross"]

        # Clean money columns: remove any non-digit characters (like $ or ,) and convert to numeric
        for column in money_cols:
            if column in df.columns:
                df[column] = pd.to_numeric(df[column].str.replace(r"[^\d.-]", "", regex=True), errors="coerce")
                
        # Clean the "Week" column: extract numbers and convert to integer
        if "Week" in df.columns:
            df["Week"] = pd.to_numeric(df["Week"].str.extract(r"(\d+)")[0], errors="coerce").astype("Int64") # extract numeric part, invalid entries → NaN, allow nullable integers
        return df # Return the cleaned DataFrame

    @staticmethod
    def clean_title_braces(title):
        """Remove {...} from movie title."""
        return re.sub(r"\{.*?\}", "", title).strip()

    @staticmethod
    def canonicalize_title(title):
        """Remove apostrophes, periods, normalize spaces into dashes."""
        transformed_title = title.replace("'", "").replace(".", "")
        transformed_title = re.sub(r"\s+", "-", transformed_title)
        transformed_title = re.sub(r"-{2,}", "-", transformed_title)
        return transformed_title.strip("-")

    @staticmethod
    def rotate_title(title):
        """Rotate first dash-separated segment to the end.
        Example: rotate_title("Back-in-the-Day") --> "in-the-Day-Back" """
        parts = title.split("-")
        if len(parts) > 1:
            return "-".join(parts[1:] + [parts[0]])
        return title
    
        
    def try_fallbacks(self, original_title, tried_urls):
        """
        Try multiple title transformations until the box office table is found.
        Returns the BeautifulSoup object and the URL used, or (None, None) if all fail.
        """
    
        # Step 1: Clean the original title
        # Remove punctuation, normalize spaces/dashes
        clean = self.canonicalize_title(original_title)
        url_candidates = [clean]  # list of possible title variations to try
    
        # Step 2: Handle "&" in titles
        # If the title contains "&" or doesn't already have "and", add a version with "&" replaced by "and"
        if "&" in clean or "and" not in clean:
            url_candidates.append(clean.replace("&", "and"))
    
        # Step 3: Generate rotated title variations
        # Example: "Back-in-the-Day" → "in-the-Day-Back"
        # Iterate over a copy of url_candidates to allow appending new variations
        for candidate in list(url_candidates):
            rotated_title = self.rotate_title(candidate)  # rotate first dash-separated word to the end
            if rotated_title not in url_candidates:
                url_candidates.append(rotated_title)  # add new variation if not already in list
    
        # Step 4: Try each candidate to construct a URL
        base_url_fmt = "https://www.the-numbers.com/movie/{title}#tab=box-office"
        for candidate in url_candidates:
            url_try = base_url_fmt.format(title=candidate)  # build full URL
    
            # Skip URLs that have already been tried
            if url_try in tried_urls:
                continue
    
            # Mark this URL as tried
            tried_urls.add(url_try)
    
            # Step 5: Attempt to fetch the page
            try:
                soup = self.fetch_page(url_try)  # fetch HTML and parse with BeautifulSoup
            except Exception:
                continue  # if fetch fails, try the next candidate
    
            # Step 6: Check if the page contains the weekend box office table
            table = self.find_weekend_table(soup)
            if table is not None:
                return soup, url_try  # success: return the page and URL used
    
        # Step 7: If no candidate worked, return None
        return None, None  # all fallbacks failed

    
        

    def scrape_all(self):
        """Main loop to scrape all URLs in self.urls."""
        results = [] # List to store final results for each movie


        for i, url in enumerate(self.urls, start=1):  # Loop through each URL with counter (i = index, url = link)
            print(f"\n--- Scraping {i}/{len(self.urls)}: {url} ---")
            tried_urls = set() # Keep track of URLs we’ve already attempted
            original_title = url.split("/movie/")[-1].split("#")[0] # Extract the title part from the URL

            # Step 1: Try the original URL
            try:
                soup = self.fetch_page(url) # Fetch the webpage and parse as BeautifulSoup
                tried_urls.add(url) # Mark as tried
                table = self.find_weekend_table(soup) # Attempt to locate the 8-column table
            except Exception:
                table = None # If the page fails to load, treat it as no table

            # Step 2: If no table found, try fallback URLs
            if table is None:
                soup, fallback_url = self.try_fallbacks(original_title, tried_urls)

                # If fallbacks also failed, record failure and skip
                if soup is None:
                    print("All fallbacks failed. Using original URL.")
                    results.append({"title": original_title, "url": url, "data": None})
                    continue
                else:
                    # Use successful fallback URL and table
                    url = fallback_url
                    table = self.find_weekend_table(soup)
                    
            # Step 3: Convert table to DataFrame
            df = self.extract_table_to_df(table)
            if df is None or df.empty:
                # Table existed but had no usable rows
                print("Table found but empty. Using original URL with no data.")
                results.append({"title": original_title, "url": url, "data": None})
                continue

            # Step 4: Clean the movie title
            cleaned_title = self.clean_title_braces(original_title)
            results.append({"title": cleaned_title, "url": url, "data": df})

        return results

    def summary_max_weeks(self, results):
        """Convert scraped results into DataFrame with movie, url, and max week."""
        summary_list = [] # List to store summary info for each movie
        
        for movie_data in results:
            df = movie_data["data"]
            max_week = df["Week"].max() if df is not None and "Week" in df.columns else None
            summary_list.append({
                "movie": movie_data["title"],
                "url": movie_data["url"],
                "max_week": max_week
            })
        return pd.DataFrame(summary_list)


# URL 91

In [7]:


scraper = BoxOfficeScraper(url_91)  # Creating instance
results = scraper.scrape_all() # scraping urls
summary_df = scraper.summary_max_weeks(results)  # extracting maximum week from each movie

print(summary_df) # printing data frame
summary_df.to_csv("box_office_summary.csv", mode='a', header=False, index=False)  #saving to csv file



--- Scraping 1/14: https://www.the-numbers.com/movie/Bugsy#tab=box-office ---

--- Scraping 2/14: https://www.the-numbers.com/movie/Cape-Fear#tab=box-office ---

--- Scraping 3/14: https://www.the-numbers.com/movie/The-Silence-of-the-Lambs#tab=box-office ---

--- Scraping 4/14: https://www.the-numbers.com/movie/The-Prince-of-Tides#tab=box-office ---

--- Scraping 5/14: https://www.the-numbers.com/movie/The-Fisher-King#tab=box-office ---

--- Scraping 6/14: https://www.the-numbers.com/movie/Thelma-&-Louise#tab=box-office ---

--- Scraping 7/14: https://www.the-numbers.com/movie/Rambling-Rose#tab=box-office ---

--- Scraping 8/14: https://www.the-numbers.com/movie/For-the-Boys#tab=box-office ---

--- Scraping 9/14: https://www.the-numbers.com/movie/Boyz-N-the-Hood#tab=box-office ---

--- Scraping 10/14: https://www.the-numbers.com/movie/JFK#tab=box-office ---

--- Scraping 11/14: https://www.the-numbers.com/movie/Beauty-and-the-Beast#tab=box-office ---

--- Scraping 12/14: https://www.t

# URL 92

In [8]:


scraper = BoxOfficeScraper(url_92) # Creating instance
results = scraper.scrape_all() # scraping urls
summary_df = scraper.summary_max_weeks(results) # extracting maximum week from each movie

print(summary_df) # printing data frame
summary_df.to_csv("box_office_summary.csv", index=False) #saving to csv file



--- Scraping 1/15: https://www.the-numbers.com/movie/Chaplin#tab=box-office ---

--- Scraping 2/15: https://www.the-numbers.com/movie/Unforgiven#tab=box-office ---

--- Scraping 3/15: https://www.the-numbers.com/movie/Scent-of-a-Woman#tab=box-office ---

--- Scraping 4/15: https://www.the-numbers.com/movie/The-Crying-Game#tab=box-office ---

--- Scraping 5/15: https://www.the-numbers.com/movie/Malcolm-X#tab=box-office ---

--- Scraping 6/15: https://www.the-numbers.com/movie/Indochine#tab=box-office ---

--- Scraping 7/15: https://www.the-numbers.com/movie/Passion-Fish#tab=box-office ---

--- Scraping 8/15: https://www.the-numbers.com/movie/Love-Field#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 9/15: https://www.the-numbers.com/movie/Lorenzo's-Oil#tab=box-office ---

--- Scraping 10/15: https://www.the-numbers.com/movie/Howards-End#tab=box-office ---

--- Scraping 11/15: https://www.the-numbers.com/movie/The-Player#tab=box-office ---

--- Scraping 12/15:

# URL 93

In [9]:
scraper = BoxOfficeScraper(url_93)
results = scraper.scrape_all()
summary_df = scraper.summary_max_weeks(results)

print(summary_df)
summary_df.to_csv("box_office_summary.csv", mode='a', header=False, index=False)



--- Scraping 1/14: https://www.the-numbers.com/movie/In-the-Name-of-the-Father#tab=box-office ---

--- Scraping 2/14: https://www.the-numbers.com/movie/What's-Love-Got-to-Do-with-It#tab=box-office ---

--- Scraping 3/14: https://www.the-numbers.com/movie/Philadelphia#tab=box-office ---

--- Scraping 4/14: https://www.the-numbers.com/movie/The-Remains-of-the-Day#tab=box-office ---

--- Scraping 5/14: https://www.the-numbers.com/movie/Schindler's-List#tab=box-office ---

--- Scraping 6/14: https://www.the-numbers.com/movie/Six-Degrees-of-Separation#tab=box-office ---

--- Scraping 7/14: https://www.the-numbers.com/movie/The-Piano#tab=box-office ---

--- Scraping 8/14: https://www.the-numbers.com/movie/Shadowlands#tab=box-office ---

--- Scraping 9/14: https://www.the-numbers.com/movie/Short-Cuts#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 10/14: https://www.the-numbers.com/movie/The-Fugitive#tab=box-office ---

--- Scraping 11/14: https://www.the-numbers.c

# URL 94

In [10]:
scraper = BoxOfficeScraper(url_94)
results = scraper.scrape_all()
summary_df = scraper.summary_max_weeks(results)

print(summary_df)
summary_df.to_csv("box_office_summary.csv", mode='a', header=False, index=False)



--- Scraping 1/15: https://www.the-numbers.com/movie/The-Shawshank-Redemption#tab=box-office ---

--- Scraping 2/15: https://www.the-numbers.com/movie/Forrest-Gump#tab=box-office ---

--- Scraping 3/15: https://www.the-numbers.com/movie/The-Madness-of-King-George#tab=box-office ---

--- Scraping 4/15: https://www.the-numbers.com/movie/Nobody's-Fool#tab=box-office ---

--- Scraping 5/15: https://www.the-numbers.com/movie/Pulp-Fiction#tab=box-office ---

--- Scraping 6/15: https://www.the-numbers.com/movie/Nell#tab=box-office ---

--- Scraping 7/15: https://www.the-numbers.com/movie/Blue-Sky#tab=box-office ---

--- Scraping 8/15: https://www.the-numbers.com/movie/Tom-&-Viv#tab=box-office ---

--- Scraping 9/15: https://www.the-numbers.com/movie/Little-Women#tab=box-office ---

--- Scraping 10/15: https://www.the-numbers.com/movie/The-Client#tab=box-office ---

--- Scraping 11/15: https://www.the-numbers.com/movie/Bullets-over-Broadway#tab=box-office ---

--- Scraping 12/15: https://www.

# URL 95

In [18]:
scraper = BoxOfficeScraper(url_95)
results = scraper.scrape_all()
summary_df = scraper.summary_max_weeks(results)

print(summary_df)
summary_df.to_csv("box_office_summary.csv", mode='a', header=False, index=False)


--- Scraping 1/14: https://www.the-numbers.com/movie/Leaving-Las-Vegas#tab=box-office ---

--- Scraping 2/14: https://www.the-numbers.com/movie/Mr.-Holland's-Opus#tab=box-office ---

--- Scraping 3/14: https://www.the-numbers.com/movie/Nixon#tab=box-office ---

--- Scraping 4/14: https://www.the-numbers.com/movie/Dead-Man-Walking#tab=box-office ---

--- Scraping 5/14: https://www.the-numbers.com/movie/The-Postman-(Il-Postino)#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 6/14: https://www.the-numbers.com/movie/Casino#tab=box-office ---

--- Scraping 7/14: https://www.the-numbers.com/movie/The-Bridges-of-Madison-County#tab=box-office ---

--- Scraping 8/14: https://www.the-numbers.com/movie/Sense-and-Sensibility#tab=box-office ---

--- Scraping 9/14: https://www.the-numbers.com/movie/Babe#tab=box-office ---

--- Scraping 10/14: https://www.the-numbers.com/movie/Braveheart#tab=box-office ---

--- Scraping 11/14: https://www.the-numbers.com/movie/Apollo-13#ta

# URL 96

In [19]:
scraper = BoxOfficeScraper(url_96)
results = scraper.scrape_all()
summary_df = scraper.summary_max_weeks(results)

print(summary_df)
summary_df.to_csv("box_office_summary.csv", mode='a', header=False, index=False)


--- Scraping 1/13: https://www.the-numbers.com/movie/Jerry-Maguire#tab=box-office ---

--- Scraping 2/13: https://www.the-numbers.com/movie/The-English-Patient#tab=box-office ---

--- Scraping 3/13: https://www.the-numbers.com/movie/The-People-vs.-Larry-Flynt#tab=box-office ---

--- Scraping 4/13: https://www.the-numbers.com/movie/Shine#tab=box-office ---

--- Scraping 5/13: https://www.the-numbers.com/movie/Sling-Blade#tab=box-office ---

--- Scraping 6/13: https://www.the-numbers.com/movie/Secrets-&-Lies#tab=box-office ---

--- Scraping 7/13: https://www.the-numbers.com/movie/Marvin's-Room#tab=box-office ---

--- Scraping 8/13: https://www.the-numbers.com/movie/Fargo#tab=box-office ---

--- Scraping 9/13: https://www.the-numbers.com/movie/Breaking-the-Waves#tab=box-office ---

--- Scraping 10/13: https://www.the-numbers.com/movie/The-Crucible#tab=box-office ---

--- Scraping 11/13: https://www.the-numbers.com/movie/Hamlet#tab=box-office ---
All fallbacks failed. Using original URL.


# URL 97

In [20]:
scraper = BoxOfficeScraper(url_97)
results = scraper.scrape_all()
summary_df = scraper.summary_max_weeks(results)

print(summary_df)
summary_df.to_csv("box_office_summary.csv", mode='a', header=False, index=False)


--- Scraping 1/15: https://www.the-numbers.com/movie/Good-Will-Hunting#tab=box-office ---

--- Scraping 2/15: https://www.the-numbers.com/movie/The-Apostle#tab=box-office ---

--- Scraping 3/15: https://www.the-numbers.com/movie/Ulee's-Gold#tab=box-office ---

--- Scraping 4/15: https://www.the-numbers.com/movie/Wag-the-Dog#tab=box-office ---

--- Scraping 5/15: https://www.the-numbers.com/movie/As-Good-as-It-Gets#tab=box-office ---

--- Scraping 6/15: https://www.the-numbers.com/movie/The-Wings-of-the-Dove#tab=box-office ---

--- Scraping 7/15: https://www.the-numbers.com/movie/Afterglow#tab=box-office ---

--- Scraping 8/15: https://www.the-numbers.com/movie/Mrs.-Brown#tab=box-office ---

--- Scraping 9/15: https://www.the-numbers.com/movie/Titanic#tab=box-office ---

--- Scraping 10/15: https://www.the-numbers.com/movie/The-Full-Monty#tab=box-office ---

--- Scraping 11/15: https://www.the-numbers.com/movie/L.A.-Confidential#tab=box-office ---
All fallbacks failed. Using original U

# URL 2015

In [21]:
scraper = BoxOfficeScraper(url_15)
results = scraper.scrape_all()
summary_df = scraper.summary_max_weeks(results)

print(summary_df)
summary_df.to_csv("box_office_summary.csv", mode='a', header=False, index=False)


--- Scraping 1/17: https://www.the-numbers.com/movie/Trumbo#tab=box-office ---

--- Scraping 2/17: https://www.the-numbers.com/movie/The-Martian#tab=box-office ---

--- Scraping 3/17: https://www.the-numbers.com/movie/The-Revenant#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 4/17: https://www.the-numbers.com/movie/Steve-Jobs#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 5/17: https://www.the-numbers.com/movie/The-Danish-Girl#tab=box-office ---

--- Scraping 6/17: https://www.the-numbers.com/movie/Carol#tab=box-office ---

--- Scraping 7/17: https://www.the-numbers.com/movie/Room#tab=box-office ---

--- Scraping 8/17: https://www.the-numbers.com/movie/Joy#tab=box-office ---

--- Scraping 9/17: https://www.the-numbers.com/movie/45-Years#tab=box-office ---

--- Scraping 10/17: https://www.the-numbers.com/movie/Brooklyn#tab=box-office ---

--- Scraping 11/17: https://www.the-numbers.com/movie/The-Big-Short#tab=box-office ---

--- 

# URL 2016

In [23]:
scraper = BoxOfficeScraper(url_16)
results = scraper.scrape_all()
summary_df = scraper.summary_max_weeks(results)

print(summary_df)
summary_df.to_csv("box_office_summary.csv", mode='a', header=False, index=False)


--- Scraping 1/16: https://www.the-numbers.com/movie/Manchester-by-the-Sea#tab=box-office ---

--- Scraping 2/16: https://www.the-numbers.com/movie/Hacksaw-Ridge#tab=box-office ---

--- Scraping 3/16: https://www.the-numbers.com/movie/La-La-Land#tab=box-office ---

--- Scraping 4/16: https://www.the-numbers.com/movie/Captain-Fantastic#tab=box-office ---

--- Scraping 5/16: https://www.the-numbers.com/movie/Fences#tab=box-office ---

--- Scraping 6/16: https://www.the-numbers.com/movie/Elle#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 7/16: https://www.the-numbers.com/movie/Loving#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 8/16: https://www.the-numbers.com/movie/Jackie#tab=box-office ---

--- Scraping 9/16: https://www.the-numbers.com/movie/Florence-Foster-Jenkins#tab=box-office ---

--- Scraping 10/16: https://www.the-numbers.com/movie/Arrival#tab=box-office ---

--- Scraping 11/16: https://www.the-numbers.com/movie/Moonlig

# URL 2017

In [31]:
scraper = BoxOfficeScraper(url_17)
results = scraper.scrape_all()
summary_df = scraper.summary_max_weeks(results)

print(summary_df)
summary_df.to_csv("box_office_summary.csv", mode='a', header=False, index=False)


--- Scraping 1/16: https://www.the-numbers.com/movie/Call-Me-by-Your-Name#tab=box-office ---

--- Scraping 2/16: https://www.the-numbers.com/movie/Phantom-Thread#tab=box-office ---

--- Scraping 3/16: https://www.the-numbers.com/movie/Get-Out#tab=box-office ---

--- Scraping 4/16: https://www.the-numbers.com/movie/Darkest-Hour#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 5/16: https://www.the-numbers.com/movie/Roman-J.-Israel,-Esq.#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 6/16: https://www.the-numbers.com/movie/The-Shape-of-Water#tab=box-office ---

--- Scraping 7/16: https://www.the-numbers.com/movie/Three-Billboards-outside-Ebbing,-Missouri#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 8/16: https://www.the-numbers.com/movie/I,-Tonya#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 9/16: https://www.the-numbers.com/movie/Lady-Bird#tab=box-office ---
All fallbacks fa

# URL 2018

In [32]:
scraper = BoxOfficeScraper(url_18)
results = scraper.scrape_all()
summary_df = scraper.summary_max_weeks(results)

print(summary_df)
summary_df.to_csv("box_office_summary.csv", mode='a', header=False, index=False)


--- Scraping 1/15: https://www.the-numbers.com/movie/Vice#tab=box-office ---

--- Scraping 2/15: https://www.the-numbers.com/movie/A-Star-Is-Born#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 3/15: https://www.the-numbers.com/movie/At-Eternity's-Gate#tab=box-office ---

--- Scraping 4/15: https://www.the-numbers.com/movie/Bohemian-Rhapsody#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 5/15: https://www.the-numbers.com/movie/Green-Book#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 6/15: https://www.the-numbers.com/movie/Roma#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 7/15: https://www.the-numbers.com/movie/The-Wife#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 8/15: https://www.the-numbers.com/movie/The-Favourite#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 9/15: https://www.the-numbers.com/movie/Can-You-Ever-F

# URL 2019

In [36]:
scraper = BoxOfficeScraper(url_19)
results = scraper.scrape_all()
summary_df = scraper.summary_max_weeks(results)

print(summary_df)
summary_df.to_csv("box_office_summary.csv", mode='a', header=False, index=False)


--- Scraping 1/15: https://www.the-numbers.com/movie/Pain-and-Glory#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 2/15: https://www.the-numbers.com/movie/Once-upon-a-Time...in-Hollywood#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 3/15: https://www.the-numbers.com/movie/Marriage-Story#tab=box-office ---

--- Scraping 4/15: https://www.the-numbers.com/movie/Joker#tab=box-office ---

--- Scraping 5/15: https://www.the-numbers.com/movie/The-Two-Popes#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 6/15: https://www.the-numbers.com/movie/Harriet#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 7/15: https://www.the-numbers.com/movie/Little-Women#tab=box-office ---

--- Scraping 8/15: https://www.the-numbers.com/movie/Bombshell#tab=box-office ---

--- Scraping 9/15: https://www.the-numbers.com/movie/Judy#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 1

# URL 2020

In [37]:
scraper = BoxOfficeScraper(url_20)
results = scraper.scrape_all()
summary_df = scraper.summary_max_weeks(results)

print(summary_df)
summary_df.to_csv("box_office_summary.csv", mode='a', header=False, index=False)


--- Scraping 1/15: https://www.the-numbers.com/movie/Sound-of-Metal#tab=box-office ---

--- Scraping 2/15: https://www.the-numbers.com/movie/Ma-Rainey's-Black-Bottom#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 3/15: https://www.the-numbers.com/movie/The-Father#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 4/15: https://www.the-numbers.com/movie/Mank#tab=box-office ---

--- Scraping 5/15: https://www.the-numbers.com/movie/Minari#tab=box-office ---

--- Scraping 6/15: https://www.the-numbers.com/movie/The-United-States-vs.-Billie-Holiday#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 7/15: https://www.the-numbers.com/movie/Pieces-of-a-Woman#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 8/15: https://www.the-numbers.com/movie/Nomadland#tab=box-office ---

--- Scraping 9/15: https://www.the-numbers.com/movie/Promising-Young-Woman#tab=box-office ---

--- Scraping 10/15: http

# URL 2021

In [33]:
scraper = BoxOfficeScraper(url_21)
results = scraper.scrape_all()
summary_df = scraper.summary_max_weeks(results)

print(summary_df)
summary_df.to_csv("box_office_summary.csv", mode='a', header=False, index=False)


--- Scraping 1/18: https://www.the-numbers.com/movie/Being-the-Ricardos#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 2/18: https://www.the-numbers.com/movie/The-Power-of-the-Dog#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 3/18: https://www.the-numbers.com/movie/tick,-tick...BOOM!#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 4/18: https://www.the-numbers.com/movie/King-Richard#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 5/18: https://www.the-numbers.com/movie/The-Tragedy-of-Macbeth#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 6/18: https://www.the-numbers.com/movie/The-Eyes-of-Tammy-Faye#tab=box-office ---

--- Scraping 7/18: https://www.the-numbers.com/movie/The-Lost-Daughter#tab=box-office ---

--- Scraping 8/18: https://www.the-numbers.com/movie/Parallel-Mothers#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scrapi